## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [7]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 模仿numpy网络结构去实现
        self.W1 = tf.Variable(tf.random.normal([28*28, 100]))
        self.b1 = tf.Variable(tf.zeros([100]))
        self.W2 = tf.Variable(tf.random.normal([100, 10]))
        self.b2 = tf.Variable(tf.zeros([10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, (-1, 28*28))
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [9]:
train_data, test_data = mnist_dataset()
for epoch in range(100):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 26.30102 ; accuracy 0.33415
epoch 1 : loss 26.009674 ; accuracy 0.33793333
epoch 2 : loss 25.7251 ; accuracy 0.34171668
epoch 3 : loss 25.447124 ; accuracy 0.34541667
epoch 4 : loss 25.175575 ; accuracy 0.34935
epoch 5 : loss 24.910305 ; accuracy 0.35316667
epoch 6 : loss 24.651054 ; accuracy 0.35688335
epoch 7 : loss 24.397558 ; accuracy 0.36026666
epoch 8 : loss 24.14957 ; accuracy 0.36361668
epoch 9 : loss 23.9068 ; accuracy 0.36715
epoch 10 : loss 23.669035 ; accuracy 0.37036666
epoch 11 : loss 23.436092 ; accuracy 0.37333333
epoch 12 : loss 23.20779 ; accuracy 0.37656668
epoch 13 : loss 22.98405 ; accuracy 0.37998334
epoch 14 : loss 22.76478 ; accuracy 0.38295
epoch 15 : loss 22.549894 ; accuracy 0.38623333
epoch 16 : loss 22.33922 ; accuracy 0.38961667
epoch 17 : loss 22.132685 ; accuracy 0.39248332
epoch 18 : loss 21.930191 ; accuracy 0.39543334
epoch 19 : loss 21.731659 ; accuracy 0.39881667
epoch 20 : loss 21.536978 ; accuracy 0.40121666
epoch 21 : loss 21.34605